In [ ]:
import numpy as np 
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from surprise import KNNWithMeans
from surprise.model_selection import cross_validate

In [3]:
df_ = pd.read_csv('/kaggle/input/amazon-ratings/ratings_Beauty.csv')
df_.head
df = df_.drop(['Timestamp'], axis = 1)

In [ ]:
ratings_matrix = df.pivot(index='UserId', columns='ProductId', values='Rating')

In [ ]:
sim_options = {'name': 'cosine', 'user_based': False}
knn_algo = KNNBasic(k=10, sim_options=sim_options)

In [ ]:
reader = Reader(rating_scale = (1,5))
data = Dataset.load_from_df(df[['UserId', 'ProductId', 'Rating']], reader)
knn = KNNBasic(k = 5)

In [ ]:
cv_results = cross_validate(knn, data, cv = 5, verbose = True)

In [ ]:
print('Average RMSE:', np.mean(cv_results['test_rmse']))
print('Average MAE:', np.mean(cv_results['test_mae']))

In [ ]:
n_clusters = 5
n_folds = 5
kmeans_algo = Kmeans(n_clusters = n_clusters, n_folds = n_folds)

In [ ]:
results = cross_validate(kmeans_algo, data, measures=['RMSE', 'MAE'], cv=n_folds, verbose=True)

In [ ]:
svd = TruncatedSVD(n_components = 10, random_state = 42)
svd.fit(ratings_matrix)

In [ ]:
user_ratings = ratings_matrix.loc[user_id, :]
user_ratings_ensemble = (0.4 * svd.transform(user_ratings.values.reshape(1, -1)) + 
                         0.3 * Kmeans.transform(user_ratings.values.reshape(1, -1)) +
                         0.3 * knn.predict(user_ratings.values.reshape(1, -1)))

# Find the top 10 products with the highest predicted ratings for the user
recommendations = pd.DataFrame(user_ratings_ensemble, index=user_ratings.index, columns=['rating'])
recommendations.sort_values('rating', ascending=False, inplace=True)
top_10_recommendations = recommendations.head(10)
print("Top 10 recommendations for user {}: \n{}".format(user_id, top_10_recommendations))